In [1]:
from data_helper import get_dataloaders_and_standarscaler_photons_from_numpy, get_photons_with_introduced_XY_symmetries
from train_helper import train_vae, train_vae_mmd
from plot_helper import plot_training_loss

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

In [2]:
# Hyperparameters
RANDOM_SEED = 123
NUM_EPOCHS = 40
LOGGING_INTERVAL=300

# RECONSTRUCTION_TERM_WEIGHT=2.8 
# LEARNING_RATE = 0.0005
# BATCH_SIZE = 1024
#TOTAL_NUM_EPOCHS
# SAVE_MODEL_FILE

PLOT_FRACTION=0.0125
TEST_FRACTION=0.4
VALIDATION_FRACTION=0.0
NUM_WORKERS=0
path='/data1/dose-3d-generative/data/training-data/PHSPs_without_VR/Filtered_E5.6_s0.0.npy'
LOAD_CHECKPOINT_PATH="/home/jakmic/Projekty/dose3d-phsp/AE_VAE/Autoencoders_Photons/TrainedModels/InfoVAE/nc_40epoch_1205_mmd_checkpoint.pth"

In [3]:
# WCZYTANIE PARAMETRÓW Z CHECKPOINTA
loaded_checkpoint = torch.load(LOAD_CHECKPOINT_PATH)

BATCH_SIZE = loaded_checkpoint["batch_size"]
RECONSTRUCTION_TERM_WEIGHT = loaded_checkpoint["reconstruction_term_weight"]
log_dict_old=loaded_checkpoint["log_dict"]

epoch = loaded_checkpoint["epoch"]
print(epoch)
TOTAL_NUM_EPOCHS=epoch+NUM_EPOCHS
model_name=loaded_checkpoint["model_name"]
print(model_name)
constrains_std_max=loaded_checkpoint["constrains_std_max"]
constrains_std_min=loaded_checkpoint["constrains_std_min"]

40
VAE_Linear_1205


In [4]:
# UTWORZENIE ŚCIEŻKI DO ZAPISU MODELU
SAVE_MODEL_FILE=f'TrainedModels/InfoVAE/nc_{TOTAL_NUM_EPOCHS}epoch_1804_mmd_checkpoint.pth'#None gdy nie tworzy się nowego zapisu

In [5]:
CUDA_DEVICE_NUM=0
DEVICE = torch.device(f'cuda:{CUDA_DEVICE_NUM}' if torch.cuda.is_available() else 'cpu')
print('Device:', DEVICE)

Device: cuda:0


In [6]:
print(torch.cuda.memory_allocated(device=DEVICE))
print(torch.cuda.memory_reserved(device=DEVICE))
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory)

8024064
8388608
Quadro RTX 8000
50959679488


In [7]:
#WCZYTANIE MODELU Z CHECKPOINTA
model = getattr(__import__('models_architecture_helper', fromlist=[model_name]), model_name)

In [8]:
#ODCZYTANIE DANYCH Z PLIKU 'photons.npy'
photons = np.load(path)

In [9]:
#USUWANIE DANYCH Z dZ MNIEJSZYM NIŻ 0
photons_nodz=np.delete(photons, np.where(photons[:,5]<0),axis=0)
print(photons_nodz.shape)

(12719035, 6)


In [10]:
#WPROWADZENIE SYMETRII X,Y DO ZBIORU FOTONÓW
symmetrized_photons=get_photons_with_introduced_XY_symmetries(photons=photons_nodz,random_seed=RANDOM_SEED)

In [11]:
df_data = pd.DataFrame(symmetrized_photons, columns = [ 'E','X', 'Y', 'dX', 'dY', 'dZ'])
df_data.head()#zawsze warto rzucić okiem na dane

,E,X,Y,dX,dY,dZ
0,4.45203,-4.39070,-2.44528,-0.169138,-0.094289,0.981072
1,0.34110,-3.09801,-3.97138,-0.119365,-0.153048,0.980983
2,1.04284,2.69596,-1.92596,0.104983,-0.074969,0.991644
3,2.84573,4.25968,-1.85673,0.164585,-0.071678,0.983755
4,0.75520,2.39109,0.23989,0.093510,0.009379,0.995574


In [12]:
train_loader, valid_loader, test_loader, stdcs = get_dataloaders_and_standarscaler_photons_from_numpy(tmp_X=symmetrized_photons,
    batch_size=BATCH_SIZE, 
    num_workers=NUM_WORKERS,
    test_fraction=TEST_FRACTION, 
    validation_fraction=VALIDATION_FRACTION)

In [13]:
# UTWORZENIE TENSORÓW Z PARAMETRAMI BRZEGOWYMI NA ODPOWIEDNIM URZĄDZENIU
if constrains_std_min is not None and constrains_std_max is not None:
    standarized_constrains_min=constrains_std_min.to(DEVICE)
    standarized_constrains_max=constrains_std_max.to(DEVICE)
else:
    standarized_constrains_min=None
    standarized_constrains_max=None


In [14]:
# model=model()
model=model(constrains_std_min=standarized_constrains_min, constrains_std_max=standarized_constrains_max)
model.to(DEVICE)
model.load_state_dict(loaded_checkpoint["model_state"])

optimizer = torch.optim.Adam(model.parameters(),
                             lr=0, 
                             weight_decay=1e-5)

optimizer.load_state_dict(loaded_checkpoint["optim_state"])

#print(optimizer.state_dict)

In [15]:
log_dict_new=train_vae_mmd(num_epochs=NUM_EPOCHS, device=DEVICE, model=model,optimizer=optimizer,train_loader=train_loader,loss_fn=None, test_loader=test_loader, logging_interval=LOGGING_INTERVAL, reconstruction_term_weight=RECONSTRUCTION_TERM_WEIGHT, save_model_file=SAVE_MODEL_FILE, total_num_of_epochs=TOTAL_NUM_EPOCHS, constrains_std_min=standarized_constrains_min, constrains_std_max=standarized_constrains_max, log_dict_old=log_dict_old)

Epoch: 001/040 | Batch 0000/0829 | Loss: 0.0318
